# Decoder layer

Ya tenemos todo lo necesario para construir el decoder

<div style="text-align:center;">
  <img src="Imagenes/transformer_architecture_model.png" alt="Multi-Head Attention" style="width:425px;height:626px;">
</div>

Pues vamos a implementarlo

## Implementación

Primero volvemos a escribir todas las clases creadas hasta ahora

In [1]:
import torch
import torch.nn as nn
import math
import torch.nn.functional as F

class ScaledDotProductAttention(nn.Module):
    def __init__(self, dim_embedding):
        """
        Args:
            dim_embedding: dimension of embedding vector
        """
        super().__init__()
        self.dim_embedding = dim_embedding
    
    def forward(self, key, query, value, mask=None):
        """
        Args:
            key: key vector
            query: query vector
            value: value vector
            mask: mask matrix (optional)
        
        Returns:
            output vector from scaled dot product attention
        """
        # MatMul
        key_trasposed = key.transpose(-1,-2)
        product = torch.matmul(query, key_trasposed)
        # scale
        scale = product / math.sqrt(self.dim_embedding)
        # Mask (optional)
        if mask is not None:
            scale = scale.masked_fill(mask == 0, float('-inf'))
        # softmax
        attention_matrix = torch.nn.functional.softmax(scale, dim=-1)
        # MatMul
        output = torch.matmul(attention_matrix, value)
        
        return output

class MultiHeadAttention(nn.Module):
    def __init__(self, heads, dim_embedding):
        """
        Args:
            heads: number of heads
            dim_embedding: dimension of embedding vector
        """
        super().__init__()
        
        self.dim_embedding = dim_embedding
        self.dim_proyection = dim_embedding // heads
        self.heads = heads
        
        self.proyection_Q = nn.Linear(dim_embedding, dim_embedding)
        self.proyection_K = nn.Linear(dim_embedding, dim_embedding)
        self.proyection_V = nn.Linear(dim_embedding, dim_embedding)
        self.attention = nn.Linear(dim_embedding, dim_embedding)

        self.scaled_dot_product_attention = ScaledDotProductAttention(self.dim_proyection)
    
    def forward(self, Q, K, V):
        """
        Args:
            Q: query vector
            K: key vector
            V: value vector

        Returns:
            output vector from multi-head attention
        """
        batch_size = Q.size(0)
        
        # perform linear operation and split into h heads
        proyection_Q = self.proyection_Q(Q).view(batch_size, -1, self.heads, self.dim_proyection)
        proyection_K = self.proyection_K(K).view(batch_size, -1, self.heads, self.dim_proyection)
        proyection_V = self.proyection_V(V).view(batch_size, -1, self.heads, self.dim_proyection)
        
        # transpose to get dimensions bs * h * sl * d_model
        proyection_Q = proyection_Q.transpose(1,2)
        proyection_K = proyection_K.transpose(1,2)
        proyection_V = proyection_V.transpose(1,2)

        # calculate attention
        scaled_dot_product_attention = self.scaled_dot_product_attention(proyection_Q, proyection_K, proyection_V)
        
        # concatenate heads and put through final linear layer
        concat = scaled_dot_product_attention.transpose(1,2).contiguous().view(batch_size, -1, self.dim_embedding)
        
        output = self.attention(concat)
    
        return output

class AddAndNorm(nn.Module):
    def __init__(self, dim_embedding):
        """
        Args:
            dim_embedding (int): Embedding dimension.
        """
        super().__init__()
        self.normalization = nn.LayerNorm(dim_embedding)

    def forward(self, x, sublayer):
        """
        Args:
            x (torch.Tensor): Input tensor.
            sublayer (torch.Tensor): Sublayer tensor.

        Returns:
            torch.Tensor: Output tensor.
        """
        return self.normalization(torch.add(x, sublayer))

class FeedForward(nn.Module):
    def __init__(self, dim_embedding, increment=4):
        super().__init__()
        self.feed_forward = nn.Sequential(
            nn.Linear(dim_embedding, dim_embedding*increment),
            nn.ReLU(),
            nn.Linear(dim_embedding*increment, dim_embedding)
        )
    
    def forward(self, x):
        """
        Args:
            x (torch.Tensor): (batch_size, seq_len, dim_embedding)

        Returns:
            torch.Tensor: (batch_size, seq_len, dim_embedding)
        """
        x = self.feed_forward(x)
        return x

Y ahora creamos la clase decoder

In [ ]:
class DecoderLayer(nn.Module):
    def __init__(self, heads, dim_embedding):
        """
        Args:
            heads: number of heads
            dim_embedding: dimension of embedding vector
        """
        super().__init__()
        self.masked_multi_head_attention = MultiHeadAttention(heads, dim_embedding)
        self.encoder_decoder_multi_head_attention = MultiHeadAttention(heads, dim_embedding)
        self.add_and_norm_1 = AddAndNorm(dim_embedding)
        self.add_and_norm_2 = AddAndNorm(dim_embedding)
        self.add_and_norm_3 = AddAndNorm(dim_embedding)
        self.feed_forward = FeedForward(dim_embedding)
    
    def forward(self, x, encoder_output, mask=None):
        """
        Args:
            x: input vector
            encoder_output: output vector from encoder
            mask: mask matrix (optional)
        
        Returns:
            output vector from decoder layer
        """
        masked_multi_head_attention = self.masked_multi_head_attention(x, x, x, mask)
        add_and_norm_1 = self.add_and_norm_1(masked_multi_head_attention, x)
        encoder_decoder_multi_head_attention = self.encoder_decoder_multi_head_attention(add_and_norm_1, encoder_output, encoder_output)
        add_and_norm_2 = self.add_and_norm_2(encoder_decoder_multi_head_attention, add_and_norm_1)
        feed_forward = self.feed_forward(add_and_norm_2)
        add_and_norm_3 = self.add_and_norm_3(feed_forward, add_and_norm_2)
        return add_and_norm_3